## 2016년도 관심 분야
* 비지도 학습 방법

In [26]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./mnist/data/', one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


## 01 기본 옵션 설정

In [0]:
total_epoch = 100 # epoch 수 설정 (왕복 횟수)
batch_size = 100 # 배치 사이즈
learning_rate = 0.0002 #학습률 (매우 느리게.)

# 신경망 레이어 구성 옵션
## 은닉층의 노드 수 결정
n_hidden = 256
# 입력층 갯수
n_input = 28*28 
# 페이크 생성기의 입력값으로 사용할 노이즈(데이터)의 크기(갯수)
n_noise = 128

## 02. 신경망 모델 구성
* 노이즈를 이용한 데이터 생성
* 비지도 학습으로 Y가 없음

In [0]:
# 판별기에 들어갈 입력
X = tf.placeholder(tf.float32, [None, n_input])

## 페이크 생성기에 들어갈 입력
Z = tf.placeholder(tf.float32, [None, n_noise])

## 생성기의 가중치(W)와 바이어스(b) 

In [0]:
# 128*256 개의 W1, 
# 128 개의 b1

#G_W1 = tf.placeholder(tf.random_normal([n_noise,n_hidden], stddev=0.01))
#G_b1 = tf.Variable(tf.zeros([n_hidden])) # 256

#G_W2 = tf.placeholder(tf.random_normal([n_hidden,n_input], stddev=0.01))
#G_b2 = tf.Variable(tf.zeros([n_input])) # 128


G_W1 = tf.Variable(tf.random_normal([128,256], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([256])) # 256

G_W2 = tf.Variable(tf.random_normal([256,784], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([784])) # 128


## 구분자의 신경망 변수 선언(W, b)
* 은닉층이 들어가면 선형 뿐만아니라 비선형 문제도 풀 수 있다.
* 은닉층이 없으면 선형문제만 풀 수 있다.

In [0]:
D_W1 = tf.Variable(tf.random_normal([784, 256], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden])) # 256


D_W2 = tf.Variable(tf.random_normal([256, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1])) # 1

## 2-2 생성기 신경망 구성

In [0]:
def generator(noise_z):
  hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1)   # 은닉층 거친 결과
  output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2) # 2번째 거친 결과(128)
  return output

### 2-3 구분자(D) 신경망 구성

In [0]:
def discriminator(inputs):
  hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
  output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
  return output

In [0]:
# def discriminator(inputs):
#   hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)   # 은닉층 거친 결과
#   output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2) # 2번째 거친 결과(128)
#   return output

### 2-4 위의 noise_z를 발생시킬 노이즈 생성 함수

In [0]:
def get_noise(batch_size, n_noise):
  return np.random.normal(size=(batch_size, n_noise))

In [0]:
#  노이즈를 이용해서 랜덤한 이미지를 생성.
G = generator(Z)

# 노이즈를 이용해서 생성한 이미지(정보)를 구분자에 넣어서 결과(판별값0~1)을 본다.
D_fake = discriminator(G)

# 진짜 이미지를 이용해서 판별한 값을 구한다.
D_real = discriminator(X)

### 3. 
* loss 
* optimizer
* 진짜 이미지를 넣었을 때, 최대값(tf.log(D_real))
* 가짜 이미지를 넣었을 때, 최대값 tf.log(1-D_fake)

In [0]:
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1-D_fake))
## 생성자의 loss
loss_G = tf.reduce_mean(tf.log(D_fake))

In [0]:
D_var_list = [D_W1, D_b1, D_W2, D_b2]  #   구분자의 변수들
G_var_list = [G_W1, G_b1, G_W2, G_b2]  #   생성자의 변수들

In [0]:
# GAN 논문의 수식에 따르면 loss 를 극대화 해야하지만, minimize 하는 최적화 함수를 사용하기 때문에
# 최적화 하려는 loss_D 와 loss_G 에 음수 부호를 붙여줍니다.
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list=G_var_list)

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0, 0

In [0]:
!mkdir samples

In [42]:
%%time
for epoch in range(total_epoch):
  for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    noise = get_noise(batch_size, n_noise)
    # 판별기와 생성기 신경망을 각각 학습시킵니다.
    _, loss_val_D = sess.run([train_D, loss_D],
    feed_dict={X: batch_xs, Z: noise})
    _, loss_val_G = sess.run([train_G, loss_G],
    feed_dict={Z: noise})
  print('Epoch:', '%04d' % epoch,
        'D loss: {:.4}'.format(loss_val_D),
        'G loss: {:.4}'.format(loss_val_G))

    #########
    # 학습이 되어가는 모습을 보기 위해 주기적으로 이미지를 생성하여 저장
    ######
  if epoch == 0 or (epoch + 1) % 10 == 0:
      sample_size = 10
      noise = get_noise(sample_size, n_noise)
      samples = sess.run(G, feed_dict={Z: noise})
      fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))
      
      for i in range(sample_size):
        ax[i].set_axis_off()
        ax[i].imshow(np.reshape(samples[i], (28, 28)))
      plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
      plt.close(fig)
print('최적화 완료!')

Epoch: 0000 D loss: -0.4419 G loss: -2.123
Epoch: 0001 D loss: -0.5463 G loss: -2.061
Epoch: 0002 D loss: -0.2765 G loss: -2.382
Epoch: 0003 D loss: -0.6085 G loss: -1.385
Epoch: 0004 D loss: -0.409 G loss: -1.917
Epoch: 0005 D loss: -0.1989 G loss: -2.478
Epoch: 0006 D loss: -0.4351 G loss: -1.949
Epoch: 0007 D loss: -0.2335 G loss: -2.938
Epoch: 0008 D loss: -0.2933 G loss: -2.675
Epoch: 0009 D loss: -0.3514 G loss: -2.558
Epoch: 0010 D loss: -0.5002 G loss: -2.551
Epoch: 0011 D loss: -0.4325 G loss: -2.124
Epoch: 0012 D loss: -0.4633 G loss: -2.083
Epoch: 0013 D loss: -0.4236 G loss: -2.46
Epoch: 0014 D loss: -0.5581 G loss: -1.909
Epoch: 0015 D loss: -0.6003 G loss: -1.842
Epoch: 0016 D loss: -0.5391 G loss: -1.778
Epoch: 0017 D loss: -0.5661 G loss: -2.032
Epoch: 0018 D loss: -0.7108 G loss: -1.825
Epoch: 0019 D loss: -0.6687 G loss: -2.032
Epoch: 0020 D loss: -0.7094 G loss: -1.841
Epoch: 0021 D loss: -0.5613 G loss: -2.177
Epoch: 0022 D loss: -0.6684 G loss: -2.009
Epoch: 0023 D